### Imports

### Arguments

In [ ]:
# see https://github.com/facebookresearch/dino/blob/main/main_dino.py get_args_parser() for explanations

class args_class:
  def __init__(self):
    # Model Parameters
    self.MOMENTUM_TEACHER = 0.996     #default 0.996
    self.OUT_DIM = 65536 #default 65536
    self.NORM_LAST_LAYER = True
    self.USE_BN_IN_HEAD = False     #default False


    # Temperature Teacher Parameters
    self.WARMUP_TEACHER_TEMP = 0.04   #default 0.04
    self.TEACHER_TEMP = 0.04   #default 0.04
    self.WARMUP_TEACHER_TEMP_EPOCHS = 30 #default 30, but erroneously default 0 in the DINO paper?

    # Training / Optimizations Parameters
    self.USE_FP16 = True #default True
    self.WEIGHT_DECAY = 0.04 #default 0.04
    self.WEIGHT_DECAY_END = 0.4 # default 0.4
    self.CLIP_GRAD = 3.0 #default 3.0
    self.BATCH_SIZE_PER_GPU = 64 #default 64
    self.EPOCHS = 100 #default 100
    self.FREEZE_LAST_LAYER = 1 #default 1
    self.LR = 0.0005 #default 0.0005
    self.WARMUP_EPOCHS = 10 #default 10
    self.MIN_LR = 1e-6 #default 1e-6
    self.optimizer = 'adamw' # default adamw    TODO:Could be constructed here? not sure.
    self.DROP_PATH_RATE = 0.1 #default 0.1

    # Multi-Crop Parameters
    self.GLOBAL_CROPS_SCALE = (0.4, 0.1) # default (0.4, 0.1)
    self.LOCAL_CROPS_NUMBER = 8 #default 8
    self.LOCAL_CROPS_SCALE = (0.05, 0.4) #default (0.05, 0.4)

    #Misc
    self.num_works = 10 # default 10
    # TODO: imgnet directory, how to store weights?

args = args_class()

### Architecture, DINOHead

### Utils

### Loss, Train one Epoch, Augmentation


In [ ]:
class DINOLoss(nn.module):
  # init vars
  # forward
    # scale student
    # center teacher
    # cross entropy
    # update center
    # return loss

In [ ]:
def train_one_epoch():
  # for it
    # update weights + lr
    # imgs to GPU
    # Forward Pass + loss

    # Update Student

    # EMA teacher

    # logging

In [ ]:
class DataAugmentationDINO(object):
  # define crops
  # make it callable

### Train

In [ ]:
def train_dino(args):
  # init rand seed
  # prep data

  # build student, teacher ???
  # add multicrop wrapper
  # init loss
  # init optimizer
  # init schedulers
  # train
    # train_one_epoch
    # write logs